# Taking Full Advantage of Parallelism With Data Distribution
_**Using Amazon SageMaker's Managed, Distributed Training with Different Data Distribution Methods**_

---

---

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
  1. [Scaling](#Scaling)
1. [Training-Hosting](#Training-Hosting)
1. [Evaluate](#Evaluate)


This notebook contains two parts of the Parallelized Data Distribution module of the Built-in Algorithms workshop. More specifically, it covers exploratory data analysis and evaluation of the trained models; training and deployment of the models is covered in the workshop lab guide.

---
# Setup

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

Let's start by specifying the S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting. In the first line of code, replace the placeholder text with your bucket name, such as `smworkshop-john smith`, with NO path. Be sure to remove the angle brackets and keep the quotes. The IAM role for S3 data access is pulled in from the SageMaker Notebook Instance.  After you have replaced the bucket name, go ahead and run the cell by clicking the 'Run cell' button in the toolbar above, or using Control + Enter from your keyboard.  Do NOT 'Run All' cells because we will be leaving the notebook for training and returning later for evaluation.

In [ ]:
bucket = '<your-bucket-name>' #  replace with bucket name in quotes only, NO URL path
prefix = 'sagemaker/data_distribution_types'

import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

Next, we'll import the Python libraries we'll need for the remainder of the exercise.

In [ ]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import io
import time
import copy
import json
import sys
import sagemaker.amazon.common as smac
import os
import warnings
warnings.simplefilter("ignore")

---

## Data

The [dataset](https://aws.amazon.com/public-datasets/gdelt/) we'll use for this notebook is from the [Global Database of Events, Language and Tone (GDELT) Project](https://www.gdeltproject.org/).  This information is freely available on S3 as part of the [AWS Public Datasets](https://aws.amazon.com/public-datasets/) program.

The data are stored as multiples files on S3, with two different formats: historical, which covers from 1979 to 2013, and daily updates which covers from 2013 on.  For this example, we'll stick to historical.  Let's bring in 1979 data for the purpose of interactive exploration.  We'll write a simple function so that later we can use it to download multiple files. Among other actions, the function downloads the data from the public S3 bucket, and reads it into a Pandas dataframe.  

In [ ]:
def get_gdelt(filename):
    s3 = boto3.resource('s3')
    s3.Bucket('gdelt-open-data').download_file('events/' + filename, '.gdelt.csv')
    df = pd.read_csv('.gdelt.csv', sep='\t')
    header = pd.read_csv('https://www.gdeltproject.org/data/lookups/CSV.header.historical.txt', sep='\t')
    df.columns = header.columns
    return df

In [ ]:
data = get_gdelt('1979.csv')
data

As we can see, there are 57 columns (you may not be able to see all 57 as they may be truncated for display purposes).  Some of the columns are sparsely populated, cryptically named, and in a format that's not particularly friendly for machine learning.  So, for our use case, we'll strip down to a few core attributes.  We'll use:

- `EventCode`: This is the raw CAMEO action code describing the action that Actor1 performed upon Actor2.  More detail can be found [here](https://www.gdeltproject.org/data/documentation/CAMEO.Manual.1.1b3.pdf)
- `NumArticles`:  This is the total number of source documents containing one or more mentions of this event. This can be used as a method of assessing the “importance” of an event; the more discussion of that event, the more likely it is to be significant
- `AvgTone`: This is the average “tone” of all documents containing one or more mentions of this event. The score ranges from -100 (extremely negative) to +100 (extremely positive). Common values range between -10 and +10, with 0 indicating neutral.
- `Actor1Geo_Lat`: This is the centroid latitude of the Actor1 landmark for mapping.
- `Actor1Geo_Long`: This is the centroid longitude of the Actor1 landmark for mapping.
- `Actor2Geo_Lat`: This is the centroid latitude of the Actor2 landmark for mapping.
- `Actor2Geo_Long`: This is the centroid longitude of the Actor2 landmark for mapping.

In [ ]:
data = data[['EventCode', 'NumArticles', 'AvgTone', 'Actor1Geo_Lat', 'Actor1Geo_Long', 'Actor2Geo_Lat', 'Actor2Geo_Long']]
data['EventCode'] = data['EventCode'].astype(object)

for column in data.select_dtypes(include=['object']).columns:
    display(pd.crosstab(index=data[column], columns='% observations', normalize='columns'))

display(data.describe())
hist = data.hist(bins=30, sharey=True, figsize=(10, 10))
plt.show()

We can see:
- `EventCode` is pretty unevenly distributed, with some events making up 7%+ of the observations and others being a thousandth of a percent.
- `AvgTone` seems to be reasonably smoothly distributed, while `NumArticles` has a long tail, and `Actor` geo features have suspiciously large spikes near 0.

Let's remove the (0, 0) lat-longs, one hot encode `EventCode`, and prepare our data for a machine learning model.  For this example we'll keep things straightforward and try to predict `AvgTone`, using the other variables in our dataset as features.

One more issue remains.  As we noticed above, some occurrences of `EventCode` are very rare, and may be unlikely to occur in every single year.  This means if we one hot encode individual years at a time, our feature matrix may change shape over the years, which will not work.  Therefore, we'll limit all years to the most common `EventCodes` from the year we current have.  Let's get this list.

In [ ]:
events = pd.crosstab(index=data['EventCode'], columns='count').sort_values(by='count', ascending=False).index[:20]

### Scaling

Now that we've explored our data and are ready to prepare for modeling, we can start developing a few simple functions to help us scale this to GDELT datasets from other years.

In [ ]:
def write_to_s3(bucket, prefix, channel, file_prefix, X, y):
    buf = io.BytesIO()
    smac.write_numpy_to_dense_tensor(buf, X.astype('float32'), y.astype('float32'))
    buf.seek(0)
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, channel, file_prefix + '.data')).upload_fileobj(buf)

def transform_gdelt(df, events=None):
    df = df[['AvgTone', 'EventCode', 'NumArticles', 'Actor1Geo_Lat', 'Actor1Geo_Long', 'Actor2Geo_Lat', 'Actor2Geo_Long']]
    df['EventCode'] = df['EventCode'].astype(object)
    if events is not None:
        df = df[np.in1d(df['EventCode'], events)]
    return pd.get_dummies(df[((df['Actor1Geo_Lat'] == 0) & (df['Actor1Geo_Long'] == 0) != True) &
                                   ((df['Actor2Geo_Lat'] == 0) & (df['Actor2Geo_Long'] == 0) != True)])
    
def prepare_gdelt(bucket, prefix, file_prefix, events=None, random_state=1729):
    df = get_gdelt(file_prefix + '.csv')
    model_data = transform_gdelt(df, events)
    train_data, validation_data = np.split(model_data.sample(frac=1, random_state=random_state).as_matrix(), 
                                           [int(0.9 * len(model_data))])
    write_to_s3(bucket, prefix, 'train', file_prefix, train_data[:, 1:], train_data[:, 0])
    write_to_s3(bucket, prefix, 'validation', file_prefix, validation_data[:, 1:], validation_data[:, 0])

In [ ]:
for year in range(1979, 1984):
    prepare_gdelt(bucket, prefix, str(year), events)

---

## Training-Hosting

Next, we'll set up training jobs and deploy the resulting models.  For those steps, please return to the workshop lab guide after the previous code cell completes running, which may take a few minutes.  You'll return to this notebook for the final evaluation steps.  If you are running the **Distributed Training** workshop module from GitHub, the link back is https://github.com/awslabs/amazon-sagemaker-workshop/tree/master/Built-in-Algorithms.

---

## Evaluate

To compare predictions from our two models, let's bring in some new data from a year the model was not trained or validated on.

In [ ]:
test_data = transform_gdelt(get_gdelt('1984.csv'), events).as_matrix()
test_X = test_data[:, 1:]
test_y = test_data[:, 0]

Now we'll need a function to convert these numpy matrices to CSVs so they can be passed to our endpoint as an HTTP POST request.

In [ ]:
def np2csv(arr):
    csv = io.BytesIO()
    np.savetxt(csv, arr, delimiter=',', fmt='%g')
    return csv.getvalue().decode().rstrip()

Next, because POST requests to our endpoint are limited to ~6MB, we'll setup a small function to split our test data up into mini-batches that are each about 5MB, loop through and invoke our endpoint to get predictions for those mini-batches, and gather them into a single array. 

In [ ]:
def predict_batches(data, endpoint):
    rows = 5. * 1024. * 1024. / sys.getsizeof(np2csv(data[0, :]))
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = []
    runtime = boto3.Session().client('runtime.sagemaker')
    for array in split_array:
        payload = np2csv(array)
        response = runtime.invoke_endpoint(EndpointName=endpoint,
                                           ContentType='text/csv',
                                           Body=payload)
        result = json.loads(response['Body'].read().decode())
        predictions += [r['score'] for r in result['predictions']]
    return np.array(predictions)

We're almost ready to get the predictions. First, however, we need to bring in the names of the endpoints we created in the previous section of the module. If you followed the naming convention in the lab guide, the names are already filled in for the code cell below. Otherwise, replace the names with the ones you chose.

In [ ]:
sharded_endpoint = 'distributed-sharded'
replicated_endpoint = 'distributed-replicated'

Now we'll compare accuracy in mean squared error (MSE).

In [ ]:
sharded_predictions = predict_batches(test_X, sharded_endpoint)
replicated_predictions = predict_batches(test_X, replicated_endpoint)

print('Sharded MSE =', np.mean((test_y - sharded_predictions) ** 2))
print('Replicated MSE =', np.mean((test_y - replicated_predictions) ** 2))

We can see that the fully replicated distribution type performs just slightly better in terms of fit.  However, this difference is small compared to the overall speedup achieved by dividing the data set into multiple S3 objects and distributing them across machines.

This concludes the notebook portion of this module.  Please return to the workshop lab guide.  If you are running the **Distributed Training** workshop module from GitHub, the link back is https://github.com/awslabs/amazon-sagemaker-workshop/tree/master/Built-in-Algorithms.